In [29]:
from pprint import pprint

import numpy as np
import pandas as pd

In [30]:
#Save dataset from pandas DataFrame
Data = {'District':['Suburban','Suburban','Rural','Urban','Urban','Urban','Rural','Suburban','Suburban','Urban','Suburban','Rural','Rural','Urban'],
        'House Type':['Detached','Detached','Detached','Semi-detached','Semi-detached','Semi-detached','Semi-detached','Terrace','Semi-detached','Terrace',
                      'Terrace','Terrace','Detached','Terrace'],
        'Income':['High','High','High','High','Low','Low','Low','High','Low','Low','Low','High','Low','High'],
        'Previous Customer':['No','Yes','No','No','No','Yes','Yes','No','No','No','Yes','Yes','No','Yes'],
        'Outcome':['Not responded','Not responded','Responded','Responded','Responded','Not responded','Responded','Not responded','Responded','Responded',
                   'Responded','Responded','Responded','Not responded']}

#data load through pandas DataFrame
dataset = pd.DataFrame(Data)
print(dataset.info())
print()
print(dataset.head(14))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   District           14 non-null     object
 1   House Type         14 non-null     object
 2   Income             14 non-null     object
 3   Previous Customer  14 non-null     object
 4   Outcome            14 non-null     object
dtypes: object(5)
memory usage: 692.0+ bytes
None

    District     House Type Income Previous Customer        Outcome
0   Suburban       Detached   High                No  Not responded
1   Suburban       Detached   High               Yes  Not responded
2      Rural       Detached   High                No      Responded
3      Urban  Semi-detached   High                No      Responded
4      Urban  Semi-detached    Low                No      Responded
5      Urban  Semi-detached    Low               Yes  Not responded
6      Rural  Semi-detached    Low        

In [31]:
#Splite data into X, Y
features = dataset[['District','House Type','Income','Previous Customer']]
target = dataset[['Outcome']]

print(features.head())

   District     House Type Income Previous Customer
0  Suburban       Detached   High                No
1  Suburban       Detached   High               Yes
2     Rural       Detached   High                No
3     Urban  Semi-detached   High                No
4     Urban  Semi-detached    Low                No


In [32]:
#Calcurate entropy Using log.
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True) #remove redundant data
    entropy = 0
    for i in range(len(elements)):
        entropy += -np.sum((counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)))
    return entropy  

#print('H(x) = ',round(entropy(target),2))

In [33]:
# Information Gain = M0 - M12 = entropy(root) - sig(weight*entropy)/N
def InfoGain(data,split_name,target_name):
    total_entropy = entropy(data[target_name]) #before split, original data entropy = MO
    #print('Entropy(D) = ', round(total_entropy, 2))
    
    # Calculate entropy_root - average(np.sum(weight*child entropy))
    vals,counts = np.unique(data[split_name],return_counts=True)

    # specific column values matchs outcome. -> if data[split_name]!=vals[i] -> save NaN Thus use dropna() method
    Weighted_Entropy = np.sum([counts[i]/np.sum(counts)*entropy(data.where(data[split_name]==vals[i]).dropna()[target_name])for i in range(len(vals))])
    #print('H(', split_name, ') = ', round(Weighted_Entropy, 2))
    
    # Calculate Information_Gain.
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain
 
print('InfoGain( District ) = ', round(InfoGain(dataset, "District", "Outcome"), 2), '\n')
print('InfoGain( House Type ) = ', round(InfoGain(dataset, "House Type", "Outcome"), 2), '\n')
print('InfoGain( Income ) = ', round(InfoGain(dataset, "Income", "Outcome"), 2), '\n')
print('InfoGain( Previous Customer ) = ', round(InfoGain(dataset, "Previous Customer", "Outcome"), 2))

InfoGain( District ) =  0.25 

InfoGain( House Type ) =  0.05 

InfoGain( Income ) =  0.15 

InfoGain( Previous Customer ) =  0.05


In [34]:
#Gain = Mo(initial dataset) - M12(Split node)
#Gain is bigger, splited node, creteria is more relevant. -> it can be root node.
#Using recursive Function. 
def decision_tree(dataset,original_data, features, target,parent_node_class=None):
    #Condition of Pause
    #1. target value is one, this must be splited.
    if len(np.unique(dataset[target])) <= 1: 
        return np.unique(dataset[target])[0]
    #2. feature = 0
    elif len(features) == 0:
        return parent_node_class
    #3. dataset =0
    elif len(dataset) == 0:
        return np.unique(original_data[target])[np.argmax(np.unique(original_data[target], return_counts=True)[1])]
    
    #Find max_infogain 
    infogain = [InfoGain(dataset, feature, target) for feature in features]

    #Save Outcome values. The meaning of node. ex) district = 'rural' -> parent_node_class saves responded.
    parent_node_class = np.unique(dataset[target])[np.argmax(np.unique(dataset[target], return_counts=True)[1])]

    #Make parent node the biggest Infogain features
    max_index = np.argmax(infogain) #Find max_InfoGain index
    parent_node = features[max_index]

    tree = {parent_node : {}}
    
    update_features = [idx for idx in features if idx != parent_node]
    
    for values in np.unique(dataset[parent_node]):
        sub_data = dataset.where(dataset[parent_node]==values).dropna()
        print(sub_data.head())
        subtree = decision_tree(sub_data,dataset,update_features,target,parent_node_class)
        print()
        tree[parent_node][values] = subtree 
        
    return (tree)

tree = decision_tree(dataset, dataset,["District","House Type","Income","Previous Customer"], "Outcome" )
pprint(tree)


   District     House Type Income Previous Customer    Outcome
2     Rural       Detached   High                No  Responded
6     Rural  Semi-detached    Low               Yes  Responded
11    Rural        Terrace   High               Yes  Responded
12    Rural       Detached    Low                No  Responded

    District     House Type Income Previous Customer        Outcome
0   Suburban       Detached   High                No  Not responded
1   Suburban       Detached   High               Yes  Not responded
7   Suburban        Terrace   High                No  Not responded
8   Suburban  Semi-detached    Low                No      Responded
10  Suburban        Terrace    Low               Yes      Responded
   District House Type Income Previous Customer        Outcome
0  Suburban   Detached   High                No  Not responded
1  Suburban   Detached   High               Yes  Not responded
7  Suburban    Terrace   High                No  Not responded

    District     House 

In [40]:
print([(feature, subtree) for feature, subtree in tree.items()])

[('District', {'Rural': 'Responded', 'Suburban': {'Income': {'High': 'Not responded', 'Low': 'Responded'}}, 'Urban': {'Previous Customer': {'No': 'Responded', 'Yes': 'Not responded'}}})]


In [38]:
#Predict Outcome for new customer.
def predict(tree, new_customer):
    for feature, subtree in tree.items():
        value = new_customer[feature]
        if value in subtree:
            if isinstance(subtree[value],dict): #if subtree is dictionary. return True
                return predict(subtree[value], new_customer)
            else:
                return subtree[value]

new_customer = {'District': 'Suburban', 'House Type': 'Detached', 'Income': 'Low', 'Previous Customer': 'Yes'}
predicted_outcome = predict(tree, new_customer)
print("Predicted Outcome is : ", predicted_outcome)

Predicted Outcome is :  Responded
